In [1]:
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


In [3]:

# Load the SciBERT model
model = SentenceTransformer('allenai/scibert_scivocab_uncased')

# Function to compute similarity between two texts
def compare_text(text1, text2):
    embeddings = model.encode([text1, text2])
    similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    return similarity_score


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [4]:

# Load data
meta = pd.read_excel('/content/meta.xlsx')
reviews = pd.read_excel('/content/reviews (1).xlsx')


In [ ]:

results = []

for index, row in reviews.iterrows():
    subfolder = row['subfolder']
    rid = row['rid']
    existing_confidence = row['confidence']

    try:
        paper_match = meta[meta['Folder Name'] == subfolder]
        if not paper_match.empty:
            paper_abstract = paper_match['abstract'].iloc[0]
            paper_title = paper_match['title'].iloc[0]
            text1 = str(paper_abstract) + ' ' + str(paper_title)
            text2 = str(row['paper_summary']) + ' ' + str(row['summary_of_strengths']) + ' ' + str(row['summary_of_weaknesses'])

            # Get SCIBERT similarity score
            bert_score = compare_text(text1, text2)

            # Convert to 1–5 scale
            if bert_score is not None:
                scaled_score = round(1 + (bert_score * 4), 2)
            else:
                scaled_score = None

            results.append({
                'subfolder': subfolder,
                'rid': rid,
                'existing_confidence': existing_confidence,
                'scibert_similarity': bert_score,
                'scaled_similarity_1_to_5': scaled_score
            })
        else:
            print(f"No matching paper found for subfolder: {subfolder}")
            results.append({
                'subfolder': subfolder,
                'rid': rid,
                'existing_confidence': existing_confidence,
                'scibert_similarity': None,
                'scaled_similarity_1_to_5': None
            })

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        results.append({
            'subfolder': subfolder,
            'rid': rid,
            'existing_confidence': existing_confidence,
            'scibert_similarity': None,
            'scaled_similarity_1_to_5': None
        })

results_df = pd.DataFrame(results)
results_df


In [7]:
# prompt: now write a code for generating corelation betwen
# correlation = results_df['existing_confidence'].corr(results_df['scaled_similarity_1_to_5'])

correlation = results_df['existing_confidence'].corr(results_df['scaled_similarity_1_to_5'])
print(f"Correlation between existing confidence and scaled similarity: {correlation}")


Correlation between existing confidence and scaled similarity: -0.05226405451437331


In [10]:
import pandas as pd

# Load the existing Excel file
reviews_df = pd.read_excel('/content/reviews_with_bert_jaccard.xlsx')

# Merge the new results with the existing dataframe
merged_df = pd.merge(reviews_df, results_df[['rid', 'scaled_similarity_1_to_5']], on='rid', how='left')

# Rename the 'scaled_similarity_1_to_5' column to 'scibert_confidence'
merged_df = merged_df.rename(columns={'scaled_similarity_1_to_5': 'scibert_confidence'})

# Save the updated dataframe to a new Excel file
merged_df.to_excel('/content/reviews_with_bert_jaccard_scibert.xlsx', index=False)

from google.colab import files
files.download('/content/reviews_with_bert_jaccard_scibert.xlsx')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>